## BERT의 문장 임베딩(SBERT)을 이용한 한국어 챗봇

SBERT를 이용하여 문장 임베딩을 얻을 수 있는 패키지인 sentence_transformers를 사용하여 쉽고 간단하게 한국어 챗봇을 구현해봅시다. 실습에 앞서 sentence_transformers를 설치합니다.
```
pip install sentence_transformers
```
트랜스포머를 이용한 한국어 챗봇 실습에서 사용했던 데이터를 그대로 사용합니다.

In [1]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import urllib.request
from sentence_transformers import SentenceTransformer

2022-04-12 17:06:18.195720: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-12 17:06:18.195755: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="data/tr_chatbot/ChatBotData.csv")
train_data = pd.read_csv('data/tr_chatbot/ChatBotData.csv')
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


문장 임베딩을 얻기 위해서 사전 훈련된 BERT를 로드합니다. 여기서는 한국어도 포함되어 학습된 다국어 모델을 로드합니다.
  
모델의 이름은 'xlm-r-100langs-bert-base-nli-stsb-mean-tokens'인데 이름이 의미하는 바는 100가지 언어를 지원(한국어 포함)하는 다국어 BERT BASE 모델로 SNLI 데이터를 학습 후 STS-B 데이터로 학습되었으며, 문장 표현을 얻기 위해서는 평균 풀링(mean-tokens)을 사용했다는 의미입니다. 다시 말해서 NLI 데이터를 학습 후에 STS 데이터로 추가 파인 튜닝한 모델이라는 의미입니다.

In [3]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

SentenceTransformer로 로드할 수 있는 다양한 모델에 대한 리스트는 아래의 링크에서 확인 가능합니다.
해당 링크에 한국어 버전의 모델들 또한 공개되어 있으니 방문해보세요.

링크 : https://huggingface.co/models?library=sentence-transformers

데이터에서 모든 질문열. 즉, train_data['Q']에 대해서 문장 임베딩 값을 구한 후 embedding이라는 새로운 열에 저장합니다.

In [4]:
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis = 1)

In [5]:
train_data[:5]

,Q,A,label,embedding
0,12시 땡!,하루가 또 가네요.,0,"[0.20179574, -0.03443809, 1.5395726, 0.0106974..."
1,1지망 학교 떨어졌어,위로해 드립니다.,0,"[0.077166, -0.034278177, 0.86244226, 0.0263605..."
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0,"[0.104452446, -0.012432261, 1.0132879, 0.02250..."
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0,"[0.09760745, -0.046716798, 0.8936941, 0.021047..."
4,PPL 심하네,눈살이 찌푸려지죠.,0,"[-0.07002868, 0.03196116, 1.4915429, 4.339431e..."


두 개의 벡터로부터 코사인 유사도를 구하는 함수 cos_sim를 정의합니다.

In [6]:
def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

return_answer 함수는 임의의 질문이 들어오면 해당 질문의 문장 임베딩 값과 챗봇 데이터의 임베딩 열. 즉, train_data['embedding']에 저장해둔 모든 질문 샘플들의 문장 임베딩 값들을 전부 비교하여 코사인 유사도 값이 가장 높은 질문 샘플을 찾아냅니다. 그리고 해당 질문 샘플과 짝이 되는 답변 샘플을 리턴합니다.

In [8]:
def return_answer(question):
    embedding = model.encode(question)
    train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    return train_data.loc[train_data['score'].idxmax()]['A']

이제 챗봇을 테스트해봅시다.

In [9]:
return_answer('결혼하고싶어')

'좋은 사람이랑 결혼할 수 있을 거예요.'

In [10]:
return_answer('나랑 커피먹을래?')

'카페인이 필요한 시간인가 봐요.'

In [13]:
return_answer('회사에 가서 뭐 해야되요?')

'잘 될거예요. 당신의 삶을 응원합니다!'